In [1]:
import cv2
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, Lambda, MaxPooling2D, Dropout, Cropping2D

Using TensorFlow backend.


In [2]:
rel_loc = "./own_data/IMG/"
info = pd.read_csv("own_data/driving_log.csv", names=['path_c', 'path_l', 'path_r', 'angle', 'throttle', 'break', 'speed'] )

In [3]:
# Change name of images

info['rel_path_c'] = info["path_c"].str.split("\\").apply(lambda x: rel_loc + str(x[-1]))
info['rel_path_l'] = info["path_l"].str.split("\\").apply(lambda x: rel_loc + str(x[-1]))
info['rel_path_r'] = info["path_r"].str.split("\\").apply(lambda x: rel_loc + str(x[-1]))

In [7]:
# Split the dataframe
train_data, val_data = train_test_split(info)

In [11]:
def resize(image):
    return cv2.resize(image, (int(1/2*320), int(1/2*160)), interpolation = cv2.INTER_CUBIC)

def recolor(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def load_data(data, batch_size=10, include_side=True):
    num_samples = data.shape[0]
    #while True:
    angles = []
    images = []
    #for offset in range(0, num_samples, batch_size):
        #for index, row in data.iloc[offset:offset+batch_size].iterrows():
    for index, row in data.iterrows():
        image = recolor(resize(cv2.imread(row["rel_path_c"])))
        image_flipped = np.fliplr(image)
        measurement = float(row["angle"])
        measurement_flipped = -measurement

        images.append(image)
        angles.append(measurement)

        images.append(image_flipped)
        angles.append(measurement_flipped)

        if include_side:
            correction = 0.2 # this is a parameter to tune
            measurement_left = measurement + correction
            measurement_right = measurement - correction
            images.append(recolor(resize(cv2.imread(row["rel_path_l"]))))
            angles.append(measurement_left)
            images.append(recolor(resize(cv2.imread(row["rel_path_r"]))))
            angles.append(measurement_right)
                
    return shuffle(np.array(images), np.array(angles))
# In the end, I didnt use generator, as AWS could fit the data into memory.


In [12]:
X_train, y_train = load_data(train_data)

In [13]:
X_val, y_val   = load_data(val_data)

In [17]:
model = Sequential()
model.add(Cropping2D(cropping=((35,15), (0,0)), input_shape=(80,160,3)))
#model.add(Lambda(lambda x: tf.image.rgb_to_grayscale(x)))
model.add(Lambda(lambda x: x/255.0-0.5))
model.add(Convolution2D(12, 5, 5, activation="relu"))
model.add(Convolution2D(24, 5, 5, activation="relu"))
model.add(Convolution2D(48, 5, 5, activation="relu"))
model.add(Convolution2D(64, 3, 3, activation="relu"))
model.add(Convolution2D(64, 3, 3, activation="relu"))

model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(50,  activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10,  activation="relu"))
model.add(Dense(1))

model.compile(loss="mse", optimizer="adam")
#model.fit_generator(train_gen,
#                    samples_per_epoch = train_data.shape[0] * 3, # This is because of the 3 camera images
#                    validation_data=val_gen,
#                    nb_val_samples= val_data.shape[0]*3, nb_epoch=10, verbose=1)

model.fit(X_train, y_train, batch_size=128, nb_epoch=5, validation_data=(X_val, y_val))

model.save("model.h5")
#exit()

Train on 16372 samples, validate on 5460 samples
Epoch 1/5
16372/16372 [==============================] - 38s - loss: 0.0320 - val_loss: 0.0240
Epoch 2/5
16372/16372 [==============================] - 35s - loss: 0.0254 - val_loss: 0.0234
Epoch 3/5
16372/16372 [==============================] - 35s - loss: 0.0229 - val_loss: 0.0225
Epoch 4/5
16372/16372 [==============================] - 35s - loss: 0.0216 - val_loss: 0.0217
Epoch 5/5
16372/16372 [==============================] - 35s - loss: 0.0209 - val_loss: 0.0210
